In [1]:
import pandas as pd
import os
import spacy
# Para spacy portugues rodar "python -m spacy download pt" no terminal
from spacy.matcher import Matcher
from time import time


In [2]:
# para usar o spacy com gpu rodar no terminal: pip install -U spacy[cuda]
spacy.prefer_gpu()

True

In [3]:
PATH_CURRENT_DIRECTORY = os.getcwd()
PATH_DATASETS = os.path.join(PATH_CURRENT_DIRECTORY,'datasets')

Os datasets serão armazenados em um dicionário cuja chave é o nome do arquivo e o valor é o dataset em si.

In [ ]:
datasets = {}
years = os.listdir(os.path.join(PATH_CURRENT_DIRECTORY, PATH_DATASETS))
files_to_read = ['licitacao.csv','item.csv']

for i, year in enumerate(years):
    if i == 0:
        for file in files_to_read:
            df = pd.read_csv(os.path.join(PATH_DATASETS,year,file))
            datasets[file.split('.')[0]] = df
    else:
        for file in files_to_read:
            df = pd.read_csv(os.path.join(PATH_DATASETS,year,file))
            datasets[file.split('.')[0]] = pd.concat([datasets[file.split('.')[0]],df], ignore_index=True)

In [5]:
datasets['licitacao'].reset_index(drop=True);
datasets['item'].reset_index(drop=True);

# Licitações

## Exploração dos dados

In [6]:
datasets['licitacao'].head()

,CD_ORGAO,NM_ORGAO,NR_LICITACAO,ANO_LICITACAO,CD_TIPO_MODALIDADE,NR_COMISSAO,ANO_COMISSAO,TP_COMISSAO,NR_PROCESSO,ANO_PROCESSO,...,TP_DOCUMENTO_FORNECEDOR,NR_DOCUMENTO_FORNECEDOR,TP_DOCUMENTO_VENCEDOR,NR_DOCUMENTO_VENCEDOR,VL_HOMOLOGADO,BL_GERA_DESPESA,DS_OBSERVACAO,PC_TX_ESTIMADA,PC_TX_HOMOLOGADA,BL_COMPARTILHADA
0,84100,PM DE ESPERANÇA DO SUL,1.0,2016,CNV,12.0,2016.0,P,001,2016.0,...,NaN,NaN,J,6233397000148.0,6420.0,S,NaN,NaN,NaN,N
1,84100,PM DE ESPERANÇA DO SUL,4.0,2016,CNV,12.0,2016.0,P,003,2016.0,...,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N
2,82701,CM DE XANGRI-LÁ,1.0,2016,PRP,12.0,2016.0,G,01,2016.0,...,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N
3,84100,PM DE ESPERANÇA DO SUL,6.0,2016,CNV,12.0,2016.0,P,005,2016.0,...,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N
4,63900,PM DE COTIPORÃ,36.0,2016,PRD,NaN,NaN,NaN,344,2016.0,...,J,9255424000171.0,NaN,NaN,NaN,S,NaN,NaN,NaN,N


In [7]:
datasets['licitacao'].shape

(237011, 61)

In [8]:
datasets['licitacao'].TP_OBJETO.value_counts()

TP_OBJETO
COM    82405
OUS    79473
CSE    54830
OSE    13730
LOC     3413
ALB     1526
CON     1020
PER      614
Name: count, dtype: int64

Apenas os processos de compra serão analisados

In [9]:
datasets['licitacao'] = datasets['licitacao'].query('TP_OBJETO == "COM"')

In [10]:
datasets['licitacao'].shape

(82405, 61)

A chave (valor unico) é composta do código do orgão (CD_ORGAO), número da licitação (NR_LICITACAO), ano da licitação (ANO_LICITACAO) e código da modalidade (CD_TIPO_MODALIDADE). Sendo assim vou criar um campo chamado ID_UNICO concatenando esses valores.

In [11]:
datasets['licitacao'].dtypes

CD_ORGAO                int64
NM_ORGAO               object
NR_LICITACAO          float64
ANO_LICITACAO           int64
CD_TIPO_MODALIDADE     object
                       ...   
BL_GERA_DESPESA        object
DS_OBSERVACAO          object
PC_TX_ESTIMADA        float64
PC_TX_HOMOLOGADA      float64
BL_COMPARTILHADA       object
Length: 61, dtype: object

In [12]:
def add_id_column(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Returns the dataset with ID_UNICO column.
    dataset must have the following columns:
    -CD_ORGAO
    -NR_LICITACAO
    -ANO_LICITACAO
    -CD_TIPO_MODALIDADE

    Example:
        df = add_id_column(df)
    """
    columns_to_concatenate = ['CD_ORGAO', 'NR_LICITACAO','ANO_LICITACAO', 'CD_TIPO_MODALIDADE']
    dataset['ID_UNICO'] = dataset[columns_to_concatenate].apply(lambda x: '_'.join(x.astype('str')), axis=1)
    return dataset

In [13]:
# Vamos considerar apenas licitações homologadas

datasets['licitacao'] = datasets['licitacao'][datasets['licitacao'].DT_HOMOLOGACAO.isnull() == False]

In [14]:
datasets['licitacao'] = add_id_column(datasets['licitacao'])

In [15]:
datasets['licitacao'].head()

,CD_ORGAO,NM_ORGAO,NR_LICITACAO,ANO_LICITACAO,CD_TIPO_MODALIDADE,NR_COMISSAO,ANO_COMISSAO,TP_COMISSAO,NR_PROCESSO,ANO_PROCESSO,...,NR_DOCUMENTO_FORNECEDOR,TP_DOCUMENTO_VENCEDOR,NR_DOCUMENTO_VENCEDOR,VL_HOMOLOGADO,BL_GERA_DESPESA,DS_OBSERVACAO,PC_TX_ESTIMADA,PC_TX_HOMOLOGADA,BL_COMPARTILHADA,ID_UNICO
1,84100,PM DE ESPERANÇA DO SUL,4.0,2016,CNV,12.0,2016.0,P,003,2016.0,...,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N,84100_4.0_2016_CNV
2,82701,CM DE XANGRI-LÁ,1.0,2016,PRP,12.0,2016.0,G,01,2016.0,...,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N,82701_1.0_2016_PRP
3,84100,PM DE ESPERANÇA DO SUL,6.0,2016,CNV,12.0,2016.0,P,005,2016.0,...,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N,84100_6.0_2016_CNV
5,53100,PM DE NOVA PALMA,3.0,2016,PRP,6157.0,2016.0,G,07,2016.0,...,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N,53100_3.0_2016_PRP
6,86800,PM DE BOA VISTA DO SUL,3.0,2016,PRP,2.0,2016.0,G,004,2016.0,...,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,N,86800_3.0_2016_PRP


In [16]:
# verifica se o campo ID_UNICO é unico

datasets['licitacao']['ID_UNICO'].value_counts().max()

1

Vamos manter apenas as colunas necessárias para analises

In [17]:
columns_to_keep = ['ID_UNICO','NM_ORGAO','DS_OBJETO','DT_ABERTURA','DT_HOMOLOGACAO']

In [18]:
datasets['licitacao'] = datasets['licitacao'][columns_to_keep]

In [19]:
datasets['licitacao'].head()

,ID_UNICO,NM_ORGAO,DS_OBJETO,DT_ABERTURA,DT_HOMOLOGACAO
1,84100_4.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de merenda escolar,2016-01-22,2016-01-29
2,82701_1.0_2016_PRP,CM DE XANGRI-LÁ,AQUISIÇÃO DE PROTOCOLADORA E FRAGMENTADORA DE ...,2016-01-26,2016-02-01
3,84100_6.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de materiais de limpeza para as dive...,2016-02-02,2016-02-11
5,53100_3.0_2016_PRP,PM DE NOVA PALMA,Aquisição de tubos de concreto,2016-02-12,2016-02-15
6,86800_3.0_2016_PRP,PM DE BOA VISTA DO SUL,Aquisição de tubos de concreto,2016-01-25,2016-01-28


Queremos explorar as compras realizadas, sendo assim os valores 'registro' da coluna DS_OBJETO serão retirados

In [21]:
# Pega a primeira palavra da coluna DS_OBJETO

datasets['licitacao']['DS_OBJETO_PRIMEIRA_PALAVRA'] = datasets['licitacao'].DS_OBJETO.apply(lambda x: x.split()[0].upper())

# Mantém apenas os registros cuja primeira palavra da coluna DS_OBJETO não começa com a palavra 'REGISTRO'
datasets['licitacao'] = datasets['licitacao'][datasets['licitacao'].DS_OBJETO_PRIMEIRA_PALAVRA != 'REGISTRO']

# ITEM

Vamos repetir o que foi feito para o arquivo de licitações

In [22]:
datasets['item'].head()

,CD_ORGAO,NR_LICITACAO,ANO_LICITACAO,CD_TIPO_MODALIDADE,NR_LOTE,NR_ITEM,NR_ITEM_ORIGINAL,DS_ITEM,QT_ITENS,SG_UNIDADE_MEDIDA,...,TP_ORCAMENTO,CD_TIPO_FAMILIA,CD_TIPO_SUBFAMILIA,TP_DOCUMENTO,NR_DOCUMENTO,TP_DOCUMENTO.1,NR_DOCUMENTO.1,TP_BENEFICIO_MICRO_EPP,PC_TX_ESTIMADA,PC_TX_HOMOLOGADA
0,77200,15.0,2016,PRP,1,2,2,"Pneu 1000/20 liso radial, dianteiro, 16 lonas,...",10.0,UN,...,M,745.0,632.0,J,8774832000177.0,NaN,NaN,L,NaN,NaN
1,77200,15.0,2016,PRP,1,4,4,"Pneu 12.4x24 garra baixa, 10 lonas",4.0,UN,...,M,745.0,632.0,J,8774832000177.0,NaN,NaN,L,NaN,NaN
2,77200,15.0,2016,PRP,1,7,7,Pneu 1400x24 mínimo 12 lonas,6.0,UN,...,M,745.0,632.0,J,8774832000177.0,NaN,NaN,L,NaN,NaN
3,77200,15.0,2016,PRP,1,8,8,"Pneu 17.5x25 12 lonas, capacidade mínima de ca...",4.0,UN,...,M,745.0,632.0,J,8774832000177.0,NaN,NaN,L,NaN,NaN
4,77200,15.0,2016,PRP,1,12,12,"Pneu 18.4x30 garra baixa, mínimo 10 lonas",4.0,UN,...,M,745.0,632.0,J,8774832000177.0,NaN,NaN,L,NaN,NaN


In [23]:
columns_to_keep = ['CD_ORGAO','NR_LICITACAO','ANO_LICITACAO','CD_TIPO_MODALIDADE','DS_ITEM','QT_ITENS','VL_UNITARIO_HOMOLOGADO', 'VL_TOTAL_HOMOLOGADO']

In [24]:
datasets['item'] = datasets['item'][columns_to_keep]

In [25]:
datasets['item'] = add_id_column(datasets['item'])

In [26]:
columns_to_keep = ['ID_UNICO','DS_ITEM','QT_ITENS','VL_UNITARIO_HOMOLOGADO', 'VL_TOTAL_HOMOLOGADO']

In [27]:
datasets['item'] = datasets['item'][columns_to_keep]

In [28]:
datasets['item'].head()

,ID_UNICO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
0,77200_15.0_2016_PRP,"Pneu 1000/20 liso radial, dianteiro, 16 lonas,...",10.0,1430.0,14300.0
1,77200_15.0_2016_PRP,"Pneu 12.4x24 garra baixa, 10 lonas",4.0,1258.0,5032.0
2,77200_15.0_2016_PRP,Pneu 1400x24 mínimo 12 lonas,6.0,1998.0,11988.0
3,77200_15.0_2016_PRP,"Pneu 17.5x25 12 lonas, capacidade mínima de ca...",4.0,2978.0,11912.0
4,77200_15.0_2016_PRP,"Pneu 18.4x30 garra baixa, mínimo 10 lonas",4.0,2500.0,10000.0


# Juntando os datasets

Nesse momento juntamos os dois arquivos com base na chave (ID_UNICO)

In [29]:
df = pd.merge(datasets['licitacao'],datasets['item'],how='inner', on='ID_UNICO')

In [30]:
df = df.reset_index(drop=True)

In [31]:
df.head()

,ID_UNICO,NM_ORGAO,DS_OBJETO,DT_ABERTURA,DT_HOMOLOGACAO,DS_OBJETO_PRIMEIRA_PALAVRA,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
0,84100_4.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de merenda escolar,2016-01-22,2016-01-29,AQUISIÇÃO,Biscoito doce maria pacote de 700 gramas,72.0,5.00,360.0
1,84100_4.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de merenda escolar,2016-01-22,2016-01-29,AQUISIÇÃO,Biscoito salgado pacote de 400 gramas,72.0,3.61,259.92
2,84100_4.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de merenda escolar,2016-01-22,2016-01-29,AQUISIÇÃO,Farinha de trigo pacote de 5kg,4.0,8.58,34.32
3,84100_4.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de merenda escolar,2016-01-22,2016-01-29,AQUISIÇÃO,Açúcar cristal pacote de 5kg,4.0,12.35,49.4
4,84100_4.0_2016_CNV,PM DE ESPERANÇA DO SUL,Aquisição de merenda escolar,2016-01-22,2016-01-29,AQUISIÇÃO,Biscoito salgado pacote de 400 gramas,60.0,3.61,216.6


Organiza a sequencia de colunas no dataset

In [32]:
columns_sequence = ['ID_UNICO', 'DT_ABERTURA', 'DT_HOMOLOGACAO', 'NM_ORGAO', 'DS_OBJETO', 'DS_ITEM', 'QT_ITENS', 'VL_UNITARIO_HOMOLOGADO', 'VL_TOTAL_HOMOLOGADO']

In [33]:
df = df[columns_sequence]

In [34]:
df.shape

(1396857, 9)

## Tipos das colunas

In [35]:
df.dtypes

ID_UNICO                   object
DT_ABERTURA                object
DT_HOMOLOGACAO             object
NM_ORGAO                   object
DS_OBJETO                  object
DS_ITEM                    object
QT_ITENS                   object
VL_UNITARIO_HOMOLOGADO    float64
VL_TOTAL_HOMOLOGADO        object
dtype: object

Altera o tipo da coluna com base no prefixo, exemplo:
- DT -> data
- DS -> string
- VL -> float

In [36]:
def change_column_type()-> None:
    """
    Altera o tipo da coluna baseado no prefixo. Ex: DT -> data, DS (descrição) -> string .

    :return: None
    """
    try:
        for col in df.columns:
            match col.split('_'):
                case ['DT',*_]:
                    df[col] = pd.to_datetime(df[col], format= 'mixed')
                case ['DS',*_]:
                    df[col] = df[col].str.upper()
                case ['VL',*_]:
                    df[col] = df[col].astype('float')
                case ['QT',*_]:
                    df[col] = df[col].astype('float')
    except Exception as error:
        print(f'Erro encontrado: {error.args[0]}')

In [37]:
change_column_type()

Erro encontrado: Out of bounds nanosecond timestamp: 0019-06-05, at position 1044


In [38]:
df.query('DT_ABERTURA == "0019-06-05"')

,ID_UNICO,DT_ABERTURA,DT_HOMOLOGACAO,NM_ORGAO,DS_OBJETO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
1323983,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,CORANTE DE MAY GRÜNWALD:a) solução para colora...,40.00,NaN,NaN
1323984,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,LÂMINA EXTENSORA OU PARA ESFREGAÇO:a) para con...,3.00,NaN,NaN
1323985,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,"TESTE PARA PESQUISA DE PCR, EM AMOSTRAS DE SOR...",9100.00,NaN,NaN
1323986,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,TESTE PARA PESQUISA DE FATOR REUMATÓIDE EM AMO...,6000.00,NaN,NaN
1323987,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,LÁPIS DERMATOGRÁFICO: a) cor: preta;b) possuir...,36.00,NaN,NaN
1323988,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,PAPEL FILTRO QUALITATIVO: a) dimensões: 50 x 5...,100.00,NaN,NaN
1323989,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,CORANTE DE HEMOGRAMA:a) compatível com o equip...,100.00,NaN,NaN
1323990,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,FRASCO COLETOR DE URINA 24 HORAS:a) corpo tran...,24.00,NaN,NaN
1323991,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,SULFATO DE BÁRIO:a) usado como contraste radio...,6.00,NaN,NaN
1323992,45000_117.0_2019_PRP,0019-06-05,2019-06-07,PM DE CAXIAS DO SUL,Fornecimento de insumos para o laboratório de ...,"ÁLCOOL ABSOLUTO 99,5%:a) álcool anidro;b) usad...",120.00,NaN,NaN


Provavelmente no preenchimento trocaram 2019 por 0019

In [39]:
df.DT_ABERTURA.replace("0019-06-05", "2019-06-05", inplace=True)

In [40]:
change_column_type()

Erro encontrado: Out of bounds nanosecond timestamp: 0219-02-28, at position 1047


In [41]:
df.DT_ABERTURA.replace("0219-02-28", "2019-02-28", inplace=True)

In [42]:
change_column_type()

Erro encontrado: Out of bounds nanosecond timestamp: 0019-05-17, at position 1047


In [43]:
df.DT_ABERTURA.replace("0019-05-17", "2019-05-17", inplace=True)

In [44]:
change_column_type()

Erro encontrado: could not convert string to float: '###############'


In [45]:
df[df.QT_ITENS == "###############"]

,ID_UNICO,DT_ABERTURA,DT_HOMOLOGACAO,NM_ORGAO,DS_OBJETO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
774622,88004_193.0_2018_PRE,2018-07-16,2018-07-19,SECRETARIA DA SAÚDE DO RS,SELEÇÃO DE FORNECEDORES PARA REGISTRO DE PREÇO...,"DIETA HIPERCAL.S/LACT,SAC,GLUTEN 1.5CAL SIST A...",###############,0.0101,28149912.0
774623,88004_193.0_2018_PRE,2018-07-16,2018-07-19,SECRETARIA DA SAÚDE DO RS,SELEÇÃO DE FORNECEDORES PARA REGISTRO DE PREÇO...,DIETA NORMOCAL SEM LACT/SAC/GLUT SIST ABERTO.,###############,0.0086,9959109.60


Provávelmente houve erro no preenchimento.Vamos calcular a quantidade provável baseada no valor total e unitário

In [ ]:
qt_erro = df[df.QT_ITENS == "###############"]
qt_erro['QT_PROVAVEL'] =  qt_erro['VL_TOTAL_HOMOLOGADO'].astype('float') / qt_erro['VL_UNITARIO_HOMOLOGADO'].astype('float')
qt_erro.QT_PROVAVEL

Esses valores parecem errados. O ideal seria confirmar as transações. Nesse momento vou apenas retirar do dataset.

In [47]:
df = df[df.QT_ITENS != "###############"]

In [48]:
change_column_type()

Erro encontrado: could not convert string to float: '###############'


O mais provável é um erro no preenchimento do campo VL_UNITARIO_HOMOLOGADO. Vou substituir o valor total pelo valor unitário e calcular o valor unitário a partir da quantidade e do valor total.
O correto nesse caso seria tentar entrar em contato com alguém que possa ajudar a conferir esse dado.

In [49]:
df.query('VL_TOTAL_HOMOLOGADO == "###############"')

,ID_UNICO,DT_ABERTURA,DT_HOMOLOGACAO,NM_ORGAO,DS_OBJETO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
181050,63000_545.0_2016_PRE,2016-06-14,2016-07-28,PM DE VIAMÃO,AQAUISIÇÃO DE GÊNEROS ALIMENTÍCIOS PARA ALIMEN...,AÇÚCAR REFINADO COM COLORAÇÃO PRÓPRIA E UNIFOR...,33662.0,139199.0,###############


No campo de valor total homologado é provavel que o valor correto seja o que está no campo de valor total unitário. O ideal seria manter esses outliers e explorar junto ao cliente.
Nesse caso não vou descartar o valor. Vou usar o valor da coluna VL_UNITARIO_HOMOLOGADO para preencher a coluna VL_TOTAL_HOMOLOGADO e então usar a quantidade para calcular o valor unitário.

In [50]:
rows_to_replace = df.query('VL_TOTAL_HOMOLOGADO == "###############"').index

In [51]:
df.QT_ITENS = df.QT_ITENS.astype('float')

In [52]:
for row in rows_to_replace:
    df.loc[row,'VL_TOTAL_HOMOLOGADO'] = df.loc[row,'VL_UNITARIO_HOMOLOGADO']
    df.loc[row,'VL_UNITARIO_HOMOLOGADO'] = df.loc[row,'VL_TOTAL_HOMOLOGADO'] / df.loc[row,'QT_ITENS']

In [53]:
df.query('VL_TOTAL_HOMOLOGADO == "###############"')

,ID_UNICO,DT_ABERTURA,DT_HOMOLOGACAO,NM_ORGAO,DS_OBJETO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO


In [54]:
change_column_type()

In [55]:
df.sort_values(by='VL_TOTAL_HOMOLOGADO', ascending = False)

,ID_UNICO,DT_ABERTURA,DT_HOMOLOGACAO,NM_ORGAO,DS_OBJETO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
643475,67300_19.0_2017_PRE,2017-12-28,2018-03-27,PM DE EUGÊNIO DE CASTRO,AQUISIÇÃO DE VEÍCULOS DE TRANSPORTE ESCOLAR DI...,ÔNIBUS RURAL ESCOLAR - ORE 3: ÔNIBUS COM COMPR...,1600.0,228912.0,366259200.0
643474,67300_19.0_2017_PRE,2017-12-28,2018-03-27,PM DE EUGÊNIO DE CASTRO,AQUISIÇÃO DE VEÍCULOS DE TRANSPORTE ESCOLAR DI...,ÔNIBUS RURAL ESCOLAR - ORE 2: ÔNIBUS COM COMPR...,1600.0,226550.0,362480000.0
643476,67300_19.0_2017_PRE,2017-12-28,2018-03-27,PM DE EUGÊNIO DE CASTRO,AQUISIÇÃO DE VEÍCULOS DE TRANSPORTE ESCOLAR DI...,ÔNIBUS RURAL ESCOLAR - ORE 1: ÔNIBUS COM COMPR...,1600.0,189900.0,303840000.0
643477,67300_19.0_2017_PRE,2017-12-28,2018-03-27,PM DE EUGÊNIO DE CASTRO,AQUISIÇÃO DE VEÍCULOS DE TRANSPORTE ESCOLAR DI...,ÔNIBUS RURAL ESCOLAR - ORE 1 (4X4): ÔNIBUS COM...,800.0,271500.0,217200000.0
178179,45100_2704.0_2016_PRP,2016-08-24,2016-08-24,PM DE CERRO LARGO,AQUISIÇÃO EMULSÃO RM1C P/ RECUPERAÇÃO DAS VIAS...,EMULSÃO ASFÁLTICA RM1C,100000.0,1989.0,198900000.0
...,...,...,...,...,...,...,...,...,...
1396088,58800_7.0_2019_PRE,2019-02-14,2019-03-27,PM DE SÃO LOURENÇO DO SUL,"AQUISIÇÃO DE FERRAMENTAS, CABOS DE AÇO E ELETR...","PLAINA MANUAL, Nº 05, PROFISSIONAL, ELÉTRICA, ...",5.0,NaN,NaN
1396304,58800_4.0_2019_PRE,2019-02-06,2019-02-12,PM DE SÃO LOURENÇO DO SUL,AQUISIÇÃO DE MATERIAL PARA DECORAÇÃO DO CARNAV...,"GRAMPOS PARA GRAMPEADORA, TAMANHO 80X10",2.0,NaN,NaN
1396305,61800_29.0_2019_PRP,2019-05-13,2019-05-21,PM DE TRÊS DE MAIO,AQUISIÇÃO DE MATERIAIS GRÁFICOS PARA USO NAS D...,FICHA DE VISITA (FEBRA AMARELA E DENGUE) BLOCO...,300.0,NaN,NaN
1396359,49200_77.0_2019_PRE,2019-05-30,2019-06-18,PM DE GUAÍBA,AQUISICAO DE MATERIAIS PARA A VIGILANCIA AMBIE...,LÁPIS DE COR 12 UNIDADES,50.0,NaN,NaN


Alguns items estão com valores que são incoerentes. Exemplo, colchão infantil à mais de 100 mil a unidade. O ideal seria conversar com uma pessoa que pudesse nos auxiliar sobre essas informações. Saber se a venda foi realizada e qual o valor real. Um outro exemplo é o registro de bermudas, onde o valor unitário registrado é de 37 mil.

In [56]:
df.dtypes

ID_UNICO                          object
DT_ABERTURA               datetime64[ns]
DT_HOMOLOGACAO            datetime64[ns]
NM_ORGAO                          object
DS_OBJETO                         object
DS_ITEM                           object
QT_ITENS                         float64
VL_UNITARIO_HOMOLOGADO           float64
VL_TOTAL_HOMOLOGADO              float64
dtype: object

## Valores nulos

In [57]:
df.isnull().sum()

ID_UNICO                       0
DT_ABERTURA                    0
DT_HOMOLOGACAO                 0
NM_ORGAO                       0
DS_OBJETO                      0
DS_ITEM                        0
QT_ITENS                       0
VL_UNITARIO_HOMOLOGADO    138422
VL_TOTAL_HOMOLOGADO       138422
dtype: int64

Os dados nulos nos campos de valores homologados podem ser descartados

In [58]:
df = df.dropna()

In [59]:
df = df.reset_index(drop=True)

# Processamento de linguagem

Vou definir uma classe para buscar os objetos comprados da coluna DS_OBJETO

In [60]:
class Phrase:
    """
    Class to handle natural language processing (npl).

    Try to find pattern in text then do data cleaning base on the EXCLUDE_LIST (values to exclude).

    """

    EXCLUDE_LIST  = ['aquisição', 'compra','registro','definição','seleção', 'pacote','gramas','preços', 'gêneros','preço','contratação', 'generos','fornecimento', 'precos','pregão', 'objeto','presente','edital','licitação', 'aquisicao', 'sistema', 'propostas', 'despesa', 'processo', 'pública','tipos','aquisições','contratacao','cotacao','fornecidos','solicitação','coleção','escolha','despesas','genêros','gêneros','compras','ãquisição', 'de gêneros', 'de preço', 'de preços', 'de generos', 'para aquisição', 'de empresa', 'de recapagens', 'para fornecimento', 'de', 'para', 'de empresa', 'para empresa', 'de expediente', 'dos preços', 'unitários', 'preços unitários']

    nlp = spacy.load("pt_core_news_lg")
    matcher = Matcher(nlp.vocab)


    def __init__(self, text:str):

        self.phrase = self.nlp(text.lower())


    def check_exclude_list(self,item_list: list)-> list:
        """

        :param item_list: List of sentenses found in phrase
        :return: input list without excluded words
        """
        return [item for item in item_list if str(item) not in self.EXCLUDE_LIST]


    def get_first_noun(self) -> list:
        """
        Gets the first noun found on phrase, except the ones in excluded_list

        :return: First noun found on phrase. If none is found returns NA
        """

        output = []
        for token in self.phrase:
            if (token.pos_ in ['NOUN','ADJ']) & (token.text not in self.EXCLUDE_LIST):
                output.append(token.text)
        if len(output)>0:
            return output
        else:
            return ['NA']


    def get_phrase_pattern(self)->list:
        """
        Finds pattern in phrase. Ex: noun followed by adjective

        :return: List of found patterns
        """

        pattern_1 = [{"POS":"NOUN"},{"POS":"ADJ"}]
        pattern_2 = [{"POS":"NOUN"},{"POS":"PROPN"}]
        pattern_3 = [{"POS":"ADP"},{"POS":"NOUN"}]

        # Priority for patterns 1 and 2

        self.matcher.add('pattern',[pattern_1,pattern_2])
        matches = self.matcher(self.phrase)
        output = []

        # if patterns 1 and 2 are not found, tries pattern 3
        if len(matches)<1:
            self.matcher.add('pattern',[pattern_3])
            matches = self.matcher(self.phrase)
            for match_id, start, end in matches:
                span = self.phrase[start:end]
                output.append(span)
            return output
        else:
            for match_id, start, end in matches:
                span = self.phrase[start:end]
                output.append(span)
            return output


    def get_phrase_object(self)->str:

        """
        Finds objects on phrase.
        Ex: ball, tires, food

        """

        output = self.get_phrase_pattern()
        output = self.check_exclude_list(output)

        if len(output)>0:
            return str(output[0][1])
        else:
            output = self.get_first_noun()
            return str(output[0])



## Aplicação no dataset

## Amostra

In [61]:
df.shape

(1258433, 9)

In [62]:
df.DS_OBJETO.unique().shape

(41173,)

In [63]:
df.DS_OBJETO.unique().shape[0]/df.shape[0]

0.032717673487583365

Vou fazer o primeiro processamento em uma amostra pequena do dataset (1%). Isso vai dar uma ideia do resultado sem a necessidade de um grande volume de processamento nos dados.

Dado que existem aproximadamente 1.200.000 registros, mas apenas cerca de 41.000 deles são únicos, não vou processar cada linha individualmente. A idéia é fazer como segue:

- Processar apenas os valores únicos
- Armazenar o resultado em um dicionário onde a chave é a frase original e o valor o resultado após processamento
- Para cada observação, buscar o valor esperado no dicionário

Com isso faremos o processamento utilizando linguagem natural de apenas `3%` do banco de dados, `economizando` tempo e recursos.

In [64]:
df_sample = df.sample(n=1500)

In [65]:
df_sample = pd.DataFrame(df_sample)

In [66]:
obj_uniques = df_sample['DS_OBJETO'].unique()
obj_uniques_size = obj_uniques.shape[0]

In [67]:
obj_size = df['DS_OBJETO'].unique().shape[0]

In [68]:
time_1 = time()
obj_dict = {}
for obj in obj_uniques:
   # obj_dict [obj] = get_phrase_object(obj)
   obj_dict [obj] = Phrase(obj).get_phrase_object()
df_sample['DS_OBJETO_RESUMIDO'] = df_sample['DS_OBJETO'].apply(lambda x: obj_dict[x])
time_2 = time()
total_time = time_2 - time_1
print(total_time)

29.632846117019653


In [69]:
# Tempo previsto em horas
obj_size/obj_uniques_size*total_time/3600

0.26664769061457516

In [70]:
df_sample.DS_OBJETO_RESUMIDO.value_counts()

DS_OBJETO_RESUMIDO
material        270
materiais       210
alimentícios    171
medicamentos    141
peças            48
               ... 
ambulatorial      1
calçados          1
construçao        1
bafômetro         1
demateriais       1
Name: count, Length: 213, dtype: int64

## Dataset completo

In [71]:
obj_uniques = df['DS_OBJETO'].unique()
obj_uniques.shape

(41173,)

In [72]:
obj_dict = {}
for obj in obj_uniques:
    obj_dict[obj] = Phrase(obj).get_phrase_object()
df['DS_OBJETO_RESUMIDO'] = df['DS_OBJETO'].apply(lambda x: obj_dict[x])

Para a coluna DS_ITEM, apenas o primeiro nome é necessário. Com isso economizamos processamento.

In [73]:
df['DS_ITEM_RESUMIDO'] = df['DS_ITEM'].apply(lambda x: x.split()[0].upper())
df.head(10)

,ID_UNICO,DT_ABERTURA,DT_HOMOLOGACAO,NM_ORGAO,DS_OBJETO,DS_ITEM,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO,DS_OBJETO_RESUMIDO,DS_ITEM_RESUMIDO
0,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,BISCOITO DOCE MARIA PACOTE DE 700 GRAMAS,72.0,5.00,360.00,merenda,BISCOITO
1,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,BISCOITO SALGADO PACOTE DE 400 GRAMAS,72.0,3.61,259.92,merenda,BISCOITO
2,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,FARINHA DE TRIGO PACOTE DE 5KG,4.0,8.58,34.32,merenda,FARINHA
3,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,AÇÚCAR CRISTAL PACOTE DE 5KG,4.0,12.35,49.40,merenda,AÇÚCAR
4,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,BISCOITO SALGADO PACOTE DE 400 GRAMAS,60.0,3.61,216.60,merenda,BISCOITO
5,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,DOCE DE LEITE POTE DE 400 GRAMAS,10.0,2.89,28.90,merenda,DOCE
6,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,ARROZ BRANCO TIPO 1 PACOTE DE 5KG,20.0,10.69,213.80,merenda,ARROZ
7,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,FARINHA DE TRIGO INTEGRAL PACOTE DE 1KG,5.0,2.49,12.45,merenda,FARINHA
8,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,GELATINA 85 GRAMAS,40.0,0.77,30.80,merenda,GELATINA
9,84100_4.0_2016_CNV,2016-01-22,2016-01-29,PM DE ESPERANÇA DO SUL,AQUISIÇÃO DE MERENDA ESCOLAR,FARINHA DE TRIGO INTEGRAL PACOTE DE 1KG,5.0,2.49,12.45,merenda,FARINHA


### Explorando as novas colunas

In [74]:
df['DS_OBJETO_RESUMIDO'].value_counts()

DS_OBJETO_RESUMIDO
material        213107
materiais       181948
alimentícios    152453
medicamentos    128192
peças            42027
                 ...  
codepas              1
principal            1
imovel               1
hidratante           1
armarios             1
Name: count, Length: 3228, dtype: int64

In [75]:
df['DS_ITEM_RESUMIDO'].value_counts()

DS_ITEM_RESUMIDO
PAPEL                21091
PNEU                 17705
FITA                 13406
FILTRO               13265
LUVA                 12597
                     ...  
GLICOSAMINA,SULF.        1
FÓRMULA/                 1
GAZIA                    1
ESTAMPAS                 1
VULCANISAÇÃO             1
Name: count, Length: 27417, dtype: int64

In [76]:
# Alterando alguns sinonimos na coluna DS_OBJETO_RESUMIDO

df.DS_OBJETO_RESUMIDO = df.DS_OBJETO_RESUMIDO.replace('materiais', 'material')
df.DS_OBJETO_RESUMIDO = df.DS_OBJETO_RESUMIDO.replace(['alimenticios', 'alimentícios', 'alimentos', 'merenda'], 'alimento')

In [77]:
df['DS_OBJETO_RESUMIDO'].value_counts()

DS_OBJETO_RESUMIDO
material        395055
alimento        200035
medicamentos    128192
peças            42027
equipamentos     36500
                 ...  
transdutor           1
codepas              1
principal            1
imovel               1
lixos                1
Name: count, Length: 3223, dtype: int64

In [78]:
# Salva o novo dataset na pasta atual
df.to_csv(os.path.join(PATH_CURRENT_DIRECTORY,'dataset.csv'), index_label=False)